In [66]:
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from sklearn.metrics import roc_auc_score

In [67]:
env = 'production_qa'

In [68]:
df = pd.read_csv('/Users/dima/hiredscore/brain_models_creation/tree_blenders/hs_tree_blenders_creation/fast_fetch/results/fetch_metrics_raw.csv')
len(df)

1971

In [52]:
df = 

array(['Negative Rejected', 'Positive Offer+ With Experience'],
      dtype=object)

In [53]:
df['is_positive'] = df['group_name'] == 'Positive Offer+ With Experience'

In [54]:
df_for_roc = df.dropna(subset=['order'])


In [56]:
df_for_roc.groupby('account_id').size()

account_id
alabama    36
indiana    26
dtype: int64

In [55]:
df_for_roc.groupby('account_id').apply(lambda d: roc_auc_score(d['is_positive'], d['order']))

account_id
alabama    0.374545
indiana    0.477124
dtype: float64

In [ ]:
reqs = {}
talents = {}


reqs_p = [ 'english_job_title', 'job_education', 
           'management_level', 'min_years_of_relevant_experience', 
           'seniority_level', 'top_category',  'sub_category']

talents_p = ['current_talent_id', 'samurai_json.total_years_of_experience', 'samurai_json.highest_degree', 'samurai_json.experience'] 
num_of_accounts = len(df['account_id'].unique())
for i, (account, account_df) in enumerate(df.groupby('account_id')):
    mongo = get_mongo_client_db(env, account)
    
    req_ids = list(account_df['req_id'].unique())
    print(f'{i}/{num_of_accounts} {account} {len(req_ids)} reqs')
    
    for r in mongo.req.find({'_id': {'$in': req_ids}}, reqs_p):
        reqs[r['_id']] = r
        
    talent_ids = list(account_df['current_talent_id'].unique())
    print(f'{i}/{num_of_accounts} {account} {len(talent_ids)} talents')
    
    for r in mongo.application.find({'current_talent_id': {'$in': talent_ids}}, talents_p):
        talents[r['current_talent_id']] = r
    

In [ ]:
for f in [ 'english_job_title', 'job_education', 'management_level', 'min_years_of_relevant_experience', 'seniority_level',
           'top_category',  'sub_category']:
    df['req_' + f] = df['req_id'].apply(lambda r: reqs[r][f])

In [ ]:
df['talent_total_years_of_experience'] = df['current_talent_id'].apply(lambda c: talents[c]['samurai_json']['total_years_of_experience'])
df['talent_highest_degree'] = df['current_talent_id'].apply(lambda c: talents[c]['samurai_json'].get('highest_degree'))

In [ ]:
def get_f(c, p, f):
    if len(c['samurai_json']['experience']) > p:
        return c['samurai_json']['experience'][p].get(f)

for p in [0, 1]:
    for f in ['title', 'seniority', 'duration_in_month']:
        df[f'talent_pos_{p}_{f}'] = df['current_talent_id'].apply(lambda c: get_f(talents[c], p , f))

In [ ]:
df.to_csv('/Users/dima/hiredscore/brain_models_creation/tree_blenders/hs_tree_blenders_creation/fast_fetch/results/fetch_metrics_raw_with_data.csv')

In [ ]:
file_path = '/Users/dima/hiredscore/brain_models_creation/tree_blenders/hs_tree_blenders_creation/fast_fetch/results/fetch_lift.xlsx'

In [ ]:
writer = pd.ExcelWriter(file_path)

In [ ]:
gb = df.groupby(['account_id','filter',  'group_name']).agg(recall=('predicted', 'mean'),
                                                            num_of_reqs=('req_id', 'nunique'), 
                                                            num_of_talents=('current_talent_id', 'count')).unstack()

gb['Fetch Lift'] = gb[('recall', 'Positive Offer+ With Experience')] / gb[('recall', 'Negative Rejected')]
gb.sort_values(['account_id', 'Fetch Lift']).to_excel(writer, sheet_name='Filters')
gb

In [ ]:
gb = df[df['filter'] == 'ALL'].groupby(['account_id', 'req_top_category', 'group_name']).agg(recall=('predicted', 'mean'),
                                                            num_of_reqs=('req_id', 'nunique'), 
                                                            num_of_talents=('current_talent_id', 'count')).unstack()

gb['Fetch Lift'] = gb[('recall', 'Positive Offer+ With Experience')] / gb[('recall', 'Negative Rejected')]
gb.sort_values(['account_id', 'Fetch Lift']).to_excel(writer, sheet_name='Top Categories')
gb

In [ ]:
gb = df[df['filter'] == 'ALL'].groupby(['account_id', 'req_seniority_level', 'group_name']).agg(recall=('predicted', 'mean'),
                                                            num_of_reqs=('req_id', 'nunique'), 
                                                            num_of_talents=('current_talent_id', 'count')).unstack()

gb['Fetch Lift'] = gb[('recall', 'Positive Offer+ With Experience')] / gb[('recall', 'Negative Rejected')]
gb.sort_values(['account_id', 'Fetch Lift']).to_excel(writer, sheet_name='Job Seniority')
gb

In [ ]:
writer.close()

In [41]:
gb = df[df['filter'] == 'ALL'].groupby(['account_id', 'group_name']).agg(recall=('predicted', 'mean'),
                                                            num_of_reqs=('req_id', 'nunique'), 
                                                            num_of_talents=('current_talent_id', 'count')).unstack()
gb

recall                                  \
group_name Negative Rejected Positive Offer+ With Experience   
account_id                                                     
alabama             0.321429                        0.100000   
indiana             0.150000                        0.333333   

                 num_of_reqs                                  \
group_name Negative Rejected Positive Offer+ With Experience   
account_id                                                     
alabama                    7                               7   
indiana                    5                               9   

              num_of_talents                                  
group_name Negative Rejected Positive Offer+ With Experience  
account_id                                                    
alabama                   56                              10  
indiana                   20                              12

In [57]:
from hs_recommendations_api_client import get_recommendations_api_client

In [58]:
api = get_recommendations_api_client('production_qa')

In [59]:
api

In [60]:
dir(api)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_get_request',
 '_put_request',
 'dismiss_dynamic_fetch_candidate',
 'environment',
 'fast_fetch',
 'fast_fetch_group_by_counts',
 'get_optimized_req_parameters',
 'http_client',
 'interactive_fetch_initial_input',
 'job_intel',
 'logger',
 'recommendations',
 'recommendations_api_endpoint',
 'recommended_reqs',
 'save_fetch_candidate',
 'save_fetch_candidates',
 'timeout']

In [65]:
dir(api.http_client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'base_uri',
 'delete',
 'extend_middleware',
 'get',
 'middleware',
 'now_iso',
 'path',
 'post',
 'put',
 'request',
 'session']

In [64]:
api.http_client.base_uri

'http://localhost'